### Model:

In [21]:
import sys
sys.path.insert(0, '../../tests')

In [22]:
from event_model import EventModel
from sgqlc.endpoint.http import HTTPEndpoint
import pandas as pd
endpoint = HTTPEndpoint('https://api.granadanet.juster.fi/v1/graphql')

In [23]:
event_data = endpoint("""
query MyQuery {
  eventByPk(id: 0) {
    betsCloseTime
    closedDynamics
    closedOracleTime
    closedRate
    createdTime
    liquidityPercent
  }
}
""")
event_data = event_data['data']['eventByPk']

In [24]:
to_timestamp = ['betsCloseTime', 'closedOracleTime', 'createdTime']
event_data.update(**{field: pd.Timestamp(event_data[field]) for field in to_timestamp})

In [25]:
deposits_raw = endpoint("""
query MyQuery {
  eventByPk(id: 0) {
    deposits(where: {}) {
      amountAboveEq
      amountBelow
      createdTime
      user {
        address
      }
      shares
      id
    }
  }
}
""")
deposits = pd.json_normalize(deposits_raw['data']['eventByPk']['deposits'])
deposits['amount'] = deposits[['amountAboveEq', 'amountBelow']].max(axis=1)

In [26]:
bets_raw = endpoint("""
query MyQuery {
  eventByPk(id: 0) {
    bets {
      amount
      createdTime
      reward
      side
      user {
        address
      }
      id
    }
  }
}
""")
bets = pd.json_normalize(bets_raw['data']['eventByPk']['bets'])

In [34]:
actions = pd.concat({
    'deposit': deposits,
    'bet': bets
}, names=['action', '_']).sort_values(['createdTime']).reset_index().drop('_', axis=1)
actions['createdTime'] = actions.createdTime.map(pd.Timestamp)

betting_time = (event_data['betsCloseTime'] - event_data['createdTime']).seconds
actions['delta_from_start'] = actions.createdTime - event_data['createdTime']
actions['time_passed'] = actions.delta_from_start.map(lambda x: x.seconds / betting_time)

actions = actions.reset_index()

# Data from granada (hide me and maybe remove):

In [35]:
from urllib.parse import urlencode
import time
import requests
import pandas as pd

def get_operations(address, last_id=None):
    method = f'https://api.granadanet.tzkt.io/v1/accounts/{address}/operations'
    params = {
        'type': 'transaction',
        'limit': '1000',
        'status': 'applied',
        'sort': '1'
    }

    if last_id:
        params['lastId'] = last_id

    data = requests.get(f'{method}?{urlencode(params)}').json()
    return data

def get_operations_deep(address, last_id=None, depth=None):
    depth = depth or 10
    data = []

    for _ in range(depth):
        new_data = get_operations(address, last_id)
        print(f'-- last_id: {last_id}, received {len(new_data)} items')
        if len(new_data) == 0:
            break

        last_id = new_data[-1]['id']
        data += new_data
        time.sleep(0.5)

    return data

def make_df(data):
    df = pd.json_normalize(data)

    df['internal'] = df['sender.address'] == 'KT1T1w1E2iMyQpogneqR2552pvrEpF7AZ8kL'
    df[ df['internal'] ]['parameter.entrypoint'].value_counts()
    df['address_counter'] = df['sender.address'] + '//' + df['counter'].astype(str)
    # unique operations:
    df['hash_counter'] = df['hash'] + '//' + df['counter'].astype(str)
    return df


def prepare_primary_calls(df):
    calls = df[ ~df['internal'] ]
    assert not any(calls['parameter.entrypoint'].isna())
    return calls

def prepare_internal_calls(df):
    internal = df[ df['internal'] ]
    return internal

def fill_internal_with_primary(internal, calls):

    # removing callbacks to make hash_counter unique key for transaction identifier:
    calls_without_callbacks = calls[
        ~calls['parameter.entrypoint'].isin(['closeCallback', 'startMeasurementCallback'])
    ]
    assert not calls_without_callbacks.hash_counter.duplicated().any()

    # adding data about source transaction to the internals:
    parent_mapping = calls_without_callbacks.set_index('hash_counter')[[
        'parameter.entrypoint', 'amount', 'parameter.value.eventId', 'parameter.value.participantAddress'
    ]]
    parent_mapping = parent_mapping.rename(columns=lambda name: f'parent.{name}')

    internal = internal.join(parent_mapping, on='hash_counter')
    assert not any(internal['parent.parameter.entrypoint'].isna())

    return internal

def check_internal_amount_stats(internal):
    # checking that close/startMeasurement is converge
    internal_amount_stats = internal.groupby('parent.parameter.entrypoint').amount.sum()

    total_bets = calls[ calls['parameter.entrypoint'] == 'bet' ]['amount'].sum()
    total_pl = calls[ calls['parameter.entrypoint'] == 'provideLiquidity' ]['amount'].sum()
    total_event_lock = calls[ calls['parameter.entrypoint'] == 'newEvent' ]['amount'].sum()

    # total_bets + total_pl + total_event_lock - internal.amount.sum()

    should_be_zero = (total_event_lock
        - internal_amount_stats['close']
        - internal_amount_stats['startMeasurement']
        - internal_amount_stats['triggerForceMajeure']
    )

    assert should_be_zero == 0

In [11]:
ADDRESS = 'KT1T1w1E2iMyQpogneqR2552pvrEpF7AZ8kL'
data = get_operations_deep(ADDRESS, depth=3)
df = make_df(data)

calls = prepare_primary_calls(df)
internal = prepare_internal_calls(df)
internal = fill_internal_with_primary(internal, calls)
check_internal_amount_stats(internal)

-- last_id: None, received 1000 items
-- last_id: 3972924, received 261 items
-- last_id: 3957114, received 0 items


In [12]:
# Then there are problem with withdrawals:
withdrawals = internal[ internal['parent.parameter.entrypoint'] == 'withdraw' ]
assert not withdrawals['parent.parameter.value.eventId'].isna().any()

bets = calls[calls['parameter.entrypoint'] == 'bet']
pls = calls[calls['parameter.entrypoint'] == 'provideLiquidity']

claims = internal[ internal['parent.parameter.entrypoint'] == 'claimRetainedProfits' ]
bets.amount.sum() + pls.amount.sum() - withdrawals.amount.sum() - claims.amount.sum()

2934694

In [13]:
bets_by_events = bets.groupby('parameter.value.eventId').amount.sum()
pls_by_events = pls.groupby('parameter.value.eventId').amount.sum()
wit_by_events = withdrawals.groupby('parent.parameter.value.eventId').amount.sum()

In [14]:
event_stats = pd.DataFrame({
    'pls': pls_by_events,
    'bets': bets_by_events,
    'withdrawals': wit_by_events
}).fillna(0)

event_stats['net_value'] = event_stats.pls + event_stats.bets - event_stats.withdrawals
event_stats[ event_stats['net_value'] != 0 ]

,pls,bets,withdrawals,net_value
0,97000000,34650000.0,129606696,2043304.0
1,50000000,22850000.0,71914716,935284.0
2,77000000,16650000.0,93618630,31370.0


In [15]:
event_stats.net_value.sum() - claims.amount.sum()

2934694.0

In [16]:
def detect_bet(bet):
    if 'Left' in bet.parameters:
        return 'aboveEq'
    elif 'Right' in bet.parameters:
        return 'below'
    else:
        raise Exception('wrong bet')

bets = bets.assign(betPool = bets.apply(detect_bet, axis=1))

In [17]:
from pytezos import pytezos
from tqdm import tqdm

address = 'KT1T1w1E2iMyQpogneqR2552pvrEpF7AZ8kL'
client = pytezos.using(
    key='../../../juster-maker/keys/tz1fvzdyC7s4mMhBrmG38kasaZjE9PHPgFEu.json',
    shell='https://rpc.tzkt.io/granadanet'
)
contract = client.contract(address)
client_address = client.key.public_key_hash()

In [18]:
event_id = '0'
event_bets = bets[ bets['parameter.value.eventId'] == event_id ]
total_bets = event_bets.groupby(['sender.address', 'betPool']).amount.sum().unstack()

event_pls = pls[ pls['parameter.value.eventId'] == event_id ]
event_withdrawals = withdrawals[ withdrawals['parent.parameter.value.eventId'] == event_id ]

event_stats = total_bets.copy()
event_stats['pls'] = event_pls.groupby('sender.address').amount.sum()
event_stats['withdrawals'] = event_withdrawals.groupby('parent.parameter.value.participantAddress').amount.sum()

# need storage state before first withdrawal:
last_storage_level = event_withdrawals.level.min() - 1
last_storage = contract.using(block_id=last_storage_level).storage

event_at_the_end = last_storage['events'][0]()

In [24]:
calls[['sender.address', 'parameter.entrypoint', 'timestamp', 'level']]

,sender.address,parameter.entrypoint,timestamp,level
1,tz1fvzdyC7s4mMhBrmG38kasaZjE9PHPgFEu,claimRetainedProfits,2021-10-05T14:36:33Z,538741
3,tz1fvzdyC7s4mMhBrmG38kasaZjE9PHPgFEu,withdraw,2021-10-05T14:34:53Z,538738
6,tz1fvzdyC7s4mMhBrmG38kasaZjE9PHPgFEu,withdraw,2021-10-05T14:31:23Z,538730
9,tz1fvzdyC7s4mMhBrmG38kasaZjE9PHPgFEu,withdraw,2021-10-05T14:31:23Z,538730
12,tz1fvzdyC7s4mMhBrmG38kasaZjE9PHPgFEu,withdraw,2021-10-05T14:31:23Z,538730
...,...,...,...,...
1256,tz1VRV2jtEVgupzgf4YoXQxK9x6Vksh9tPdM,provideLiquidity,2021-08-23T16:38:35Z,370979
1257,tz1RorGpGL42VEkd3zRBK5s8uAk3uuismC2y,provideLiquidity,2021-08-23T16:38:35Z,370979
1258,tz1UZbnprFkQe4qR11m8MFvs19JuEN3r8rG6,newEvent,2021-08-23T16:37:00Z,370975
1259,tz1fvzdyC7s4mMhBrmG38kasaZjE9PHPgFEu,newEvent,2021-08-23T16:37:00Z,370975


In [22]:
calls.iloc[0]

type                                                                          transaction
id                                                                                6750900
level                                                                              538741
timestamp                                                            2021-10-05T14:36:33Z
block                                   BL1NHgjvTiRJy6D6mmEhNtU7QinF8xd2T2rDyMtdSUMeeT...
hash                                    onkBYuw49AnThFvvChgjENbxXyMNBZWTdxBS6q8mJrZXr2...
counter                                                                            413424
nonce                                                                                 NaN
gasLimit                                                                            15000
gasUsed                                                                             10957
storageLimit                                                                          100
storageUse

In [19]:
actions

,action,amountAboveEq,amountBelow,createdTime,shares,id,user.address,amount,reward,side,delta_from_start,time_passed
0,deposit,1.000000,1.000000,2021-08-23 16:38:35+00:00,100.000000,3,tz1iosWvYEaNdFXC7w6wcn5rFFfFuWaKUMVK,1.00,NaN,NaN,0 days 00:01:35,0.001101
1,bet,NaN,NaN,2021-08-23 16:40:05+00:00,NaN,1,tz1UZbnprFkQe4qR11m8MFvs19JuEN3r8rG6,1.00,1.499998,ABOVE_EQ,0 days 00:03:05,0.002144
2,bet,NaN,NaN,2021-08-23 16:40:20+00:00,NaN,3,tz1iosWvYEaNdFXC7w6wcn5rFFfFuWaKUMVK,0.10,0.433330,BELOW,0 days 00:03:20,0.002318
3,bet,NaN,NaN,2021-08-23 16:41:25+00:00,NaN,5,tz1VRV2jtEVgupzgf4YoXQxK9x6Vksh9tPdM,0.25,0.833860,BELOW,0 days 00:04:25,0.003071
4,deposit,2.000000,0.720001,2021-08-23 16:41:25+00:00,119.999760,6,tz1UZbnprFkQe4qR11m8MFvs19JuEN3r8rG6,2.00,NaN,NaN,0 days 00:04:25,0.003071
...,...,...,...,...,...,...,...,...,...,...,...,...
90,bet,NaN,NaN,2021-08-23 17:24:05+00:00,NaN,150,tz1UZbnprFkQe4qR11m8MFvs19JuEN3r8rG6,0.10,0.197601,BELOW,0 days 00:47:05,0.032742
91,deposit,1.950027,2.000000,2021-08-23 17:24:20+00:00,197.481169,51,tz1VRV2jtEVgupzgf4YoXQxK9x6Vksh9tPdM,2.00,NaN,NaN,0 days 00:47:20,0.032916
92,bet,NaN,NaN,2021-08-23 17:24:50+00:00,NaN,153,tz1fvzdyC7s4mMhBrmG38kasaZjE9PHPgFEu,1.00,2.014518,ABOVE_EQ,0 days 00:47:50,0.033264
93,bet,NaN,NaN,2021-08-23 17:25:20+00:00,NaN,154,tz1iosWvYEaNdFXC7w6wcn5rFFfFuWaKUMVK,0.10,0.199523,BELOW,0 days 00:48:20,0.033611


# Model:

In [36]:
# resorting:
actions.loc[ actions.index == 19, 'index' ] = 17.5
actions.loc[ actions.index == 22, 'index' ] = 20.5
actions.loc[ actions.index == 32, 'index' ] = 29.5
# actions.loc[ actions.index == 34, 'index' ] = 32.5
actions = actions.sort_values('index')

In [37]:
event = EventModel(winning_pool='below', fee=event_data['liquidityPercent'])

for _, action in actions.iterrows():
    amount = action.amount*10**6
    pool_a = event.to_dict()['pool_a'] / 10**6
    pool_b = event.to_dict()['pool_b'] / 10**6
    general_info = f'{pool_a:5.2f}:{pool_b:5.2f} | {action.time_passed:10.8f} | ({action.name})'

    if action.action == 'bet':
        side = 'aboveEq' if action.side == 'ABOVE_EQ' else 'below'
        reward = (amount + event.calc_bet_profit(amount, side, action.time_passed)) / 10**6
        event.bet(
            action['user.address'],
            amount,
            side,
            action.time_passed)

        divergence = action.reward / reward - 1
        action_side = f'{action.action}:{side}'
        print(f'{action_side:14s}: {action.amount:5.2f}, '
              + f'reward contract: {action.reward:6.2f}, reward model: {reward:6.2f}, '
              + f'divergence: {divergence*100:5.2f}%, {general_info}')

    elif action.action == 'deposit':
        a = int(action.amountAboveEq)
        b = int(action.amountBelow)
        a, b, shares = event.calc_provided_shares(amount, a, b)
        shares /= 10**6
        event.provide_liquidity(
            action['user.address'],
            amount,
            a,
            b)

        divergence = action.shares / shares - 1
        print(f'{action.action:14s}: {action.amount:5.2f}, '
              + f'shares contract: {action.shares:6.2f}, shares model: {shares:6.2f}, '
              + f'divergence: {divergence*100:5.2f}%, {general_info}')

    # print(f'{action.action:10s}: {action.amount:.3f}, A: {a:2.2f}, B: {b:2.2f}')

deposit       :  1.00, shares contract: 100.00, shares model: 100.00, divergence:  0.00%,  0.00: 0.00 | 0.00110107 | (0)
bet:aboveEq   :  1.00, reward contract:   1.50, reward model:   1.50, divergence:  0.00%,  1.00: 1.00 | 0.00214418 | (1)
bet:below     :  0.10, reward contract:   0.43, reward model:   0.43, divergence:  0.01%,  2.00: 0.50 | 0.00231803 | (2)
deposit       :  2.00, shares contract: 120.00, shares model: 120.00, divergence:  0.00%,  1.67: 0.60 | 0.00307140 | (3)
bet:below     :  0.25, reward contract:   0.83, reward model:   0.83, divergence:  0.01%,  3.67: 1.32 | 0.00307140 | (4)
bet:below     :  0.25, reward contract:   0.67, reward model:   0.67, divergence:  0.01%,  3.08: 1.57 | 0.00376681 | (5)
bet:aboveEq   :  0.50, reward contract:   0.79, reward model:   0.79, divergence:  0.00%,  2.66: 1.82 | 0.00446222 | (6)
deposit       :  5.00, shares contract: 348.17, shares model: 348.16, divergence:  0.00%,  3.16: 1.53 | 0.00463607 | (7)
bet:aboveEq   :  0.10, reward co

In [11]:
sum(event.diffs.values())

-6.0

In [12]:
len(actions)

95

In [151]:
positions_raw = endpoint("""
query MyQuery {
  eventByPk(id: 0) {
    positions {
      liquidityProvidedAboveEq
      liquidityProvidedBelow
      rewardAboveEq
      rewardBelow
      shares
      user {
        address
      }
      value
      withdrawn
    }
  }
}
""")

positions = pd.json_normalize(positions_raw['data']['eventByPk']['positions']).set_index('user.address')

In [152]:
positions

,liquidityProvidedAboveEq,liquidityProvidedBelow,rewardAboveEq,rewardBelow,shares,value,withdrawn
user.address,,,,,,,
tz1fvzdyC7s4mMhBrmG38kasaZjE9PHPgFEu,11.659768,11.918189,7.128656,4.738721,1178.567848,16.734450,True
tz1UZbnprFkQe4qR11m8MFvs19JuEN3r8rG6,16.508256,15.720001,6.264887,9.845787,1594.998435,25.748526,True
tz1iosWvYEaNdFXC7w6wcn5rFFfFuWaKUMVK,21.470987,21.905145,6.991745,6.655733,2168.464873,28.685992,True
tz1VRV2jtEVgupzgf4YoXQxK9x6Vksh9tPdM,18.670077,18.590943,8.059280,9.883231,1862.376105,28.436689,True
tz1RorGpGL42VEkd3zRBK5s8uAk3uuismC2y,25.975815,24.405200,5.376724,5.180865,2495.248004,30.063183,True


In [157]:
positions.shares - pd.Series(event.shares) / 10**6

tz1RorGpGL42VEkd3zRBK5s8uAk3uuismC2y    30.445696
tz1UZbnprFkQe4qR11m8MFvs19JuEN3r8rG6    28.039040
tz1VRV2jtEVgupzgf4YoXQxK9x6Vksh9tPdM     8.640419
tz1fvzdyC7s4mMhBrmG38kasaZjE9PHPgFEu    36.919466
tz1iosWvYEaNdFXC7w6wcn5rFFfFuWaKUMVK    27.884329
dtype: float64

In [ ]:
event.diffs

In [161]:
event.provide_liquidity('a', 10_000_000, 1, 1)

<Event>
{
    "pool_a": 99791222.0,
    "pool_b": 103304722.0,
    "total_shares": 10150283908,
    "fee": 0.025,
    "winning_pool": "below",
    "diffs": {
        "tz1UZbnprFkQe4qR11m8MFvs19JuEN3r8rG6": 999839.0,
        "tz1iosWvYEaNdFXC7w6wcn5rFFfFuWaKUMVK": -293012.0,
        "tz1VRV2jtEVgupzgf4YoXQxK9x6Vksh9tPdM": 1481168.0,
        "tz1fvzdyC7s4mMhBrmG38kasaZjE9PHPgFEu": -799723.0,
        "tz1RorGpGL42VEkd3zRBK5s8uAk3uuismC2y": -1388278.0
    },
    "shares": {
        "tz1iosWvYEaNdFXC7w6wcn5rFFfFuWaKUMVK": 2140580544,
        "tz1UZbnprFkQe4qR11m8MFvs19JuEN3r8rG6": 1566959395,
        "tz1RorGpGL42VEkd3zRBK5s8uAk3uuismC2y": 2464802308,
        "tz1VRV2jtEVgupzgf4YoXQxK9x6Vksh9tPdM": 1853735686,
        "tz1fvzdyC7s4mMhBrmG38kasaZjE9PHPgFEu": 1141648382,
        "a": 982557593
    }
}